In [1]:
"""
Description: Script to optimize Duca's currency mix.
Author: Jeroen van Dijk & Victor de Graaff
Date: 04-11-2020
Maintainer: Jeroen van Dijk & Victor de Graaff
Email: jeroen.vandijk@d-data.nl & victor.degraaff@d-data.nl
Status: Dev
"""

"\nDescription: Script to optimize Duca's currency mix.\nAuthor: Jeroen van Dijk & Victor de Graaff\nDate: 04-11-2020\nMaintainer: Jeroen van Dijk & Victor de Graaff\nEmail: jeroen.vandijk@d-data.nl & victor.degraaff@d-data.nl\nStatus: Dev\n"

In [2]:
# Load common imports
%run ./CommonImports.ipynb

In [3]:
# Load common functions and currencies lists:
# - all_currencies
# - obsolete_currencies
# - p13_currencies
# - f_currencies
# - ff_currencies
# - currencies_per_continent
%run ./Utilities.ipynb

In [4]:
# Load loss functions
# - calculate_loss_function_around_one(weights)
# - calculate_loss_function_vs_t_minus_one(weights)
# - calculate_loss_function_vs_t_minus_one_with_balancing(weights)
# - calculate_loss_function_vs_t_minus_one_for_period(weights, max_date)
%run ./LossFunctions.ipynb

In [5]:
from scipy.stats import pearsonr

currency_pairs = [(c1, c2) for c1 in all_currencies for c2 in all_currencies if c1 > c2]

highly_correlated_currencies = set()
highly_unstable_currencies = set()

for base_currency in all_currencies:
    print(f"Evaluating correlations using base currency {base_currency}")
    df = create_original_df(base_currency)
    
    median = np.median(np.array([pearsonr(df[c1], df[c2])[0] 
                                 for c1, c2 in currency_pairs 
                                 if c1 != base_currency and c2 != base_currency]))
    
    if median > .9:
        highly_unstable_currencies.add(base_currency)
        continue

    for c1, c2 in currency_pairs:
        if base_currency == c1 or base_currency == c2:
            continue
            
        if c1 in highly_correlated_currencies or c2 in highly_correlated_currencies:
            continue
            
        corr = pearsonr(df[c1], df[c2])[0]

        if abs(corr) > .98:
            if trading_volumes[c1] >= trading_volumes[c2] and \
               trading_volumes[base_currency] >= trading_volumes[c2]:
                print(f"According to {base_currency} {c1} and {c2} are highly corrolated:", corr)
                highly_correlated_currencies.add(c2)
            elif trading_volumes[c2] >= trading_volumes[c1] and \
                 trading_volumes[base_currency] >= trading_volumes[c1]:
                print(f"According to {base_currency} {c2} and {c1} are highly corrolated:", corr)
                highly_correlated_currencies.add(c1)

    print(highly_unstable_currencies, highly_correlated_currencies)
    
highly_unstable_currencies, highly_correlated_currencies

Evaluating correlations using base currency EUR
According to EUR USD and HKD are highly corrolated: 0.999674208784576
set() {'HKD'}
Evaluating correlations using base currency HRK
set() {'HKD'}
Evaluating correlations using base currency NOK
According to NOK EUR and DKK are highly corrolated: 0.9998690042934141
According to NOK EUR and HRK are highly corrolated: 0.9868802261131508
set() {'HKD', 'HRK', 'DKK'}
Evaluating correlations using base currency GBP
set() {'HKD', 'HRK', 'DKK'}
Evaluating correlations using base currency PLN
set() {'HKD', 'HRK', 'DKK'}
Evaluating correlations using base currency DKK
set() {'HKD', 'HRK', 'DKK'}
Evaluating correlations using base currency RON
set() {'HKD', 'HRK', 'DKK'}
Evaluating correlations using base currency KRW
set() {'HKD', 'HRK', 'DKK'}
Evaluating correlations using base currency CHF
According to CHF EUR and RON are highly corrolated: 0.9862247598865075
According to CHF NOK and RUB are highly corrolated: 0.9802535850097623
set() {'HKD', 'HRK

({'TRY', 'ZAR'}, {'DKK', 'HKD', 'HRK', 'MYR', 'PHP', 'RON', 'RUB', 'THB'})

In [6]:
# Create dataframe with pre-defined base currency
df = create_original_df("EUR")

In [14]:
pd.date_range(start='2005-04-01', periods=40, freq='3MS')

DatetimeIndex(['2005-04-01', '2005-07-01', '2005-10-01', '2006-01-01', '2006-04-01', '2006-07-01', '2006-10-01', '2007-01-01', '2007-04-01', '2007-07-01', '2007-10-01', '2008-01-01', '2008-04-01', '2008-07-01', '2008-10-01', '2009-01-01', '2009-04-01', '2009-07-01', '2009-10-01', '2010-01-01', '2010-04-01', '2010-07-01', '2010-10-01', '2011-01-01', '2011-04-01', '2011-07-01', '2011-10-01', '2012-01-01', '2012-04-01', '2012-07-01', '2012-10-01', '2013-01-01', '2013-04-01', '2013-07-01', '2013-10-01', '2014-01-01', '2014-04-01', '2014-07-01', '2014-10-01', '2015-01-01'], dtype='datetime64[ns]', freq='3MS')

In [26]:
# Determine splits for dev- (train and test) and validation set
train_start = date(2005, 4, 1)
test_start = date(2018, 1, 1)
validation_start = date(2018, 1, 1)
validation_end = date(2020, 10, 1)

# Create dev (train and test) and validation set
train, test, dev, val = split_data(df, train_start, test_start, validation_start, validation_end)

candidate_currencies = [c for c in all_currencies if c not in highly_correlated_currencies and
                                                     c not in highly_unstable_currencies]
more_stable_currencies = [c for c in all_currencies if c not in highly_unstable_currencies]
candidate_currencies.sort()

In [20]:
def run_optimization(selected_currencies, 
                     max_iter=100, 
                     loss_function=calculate_loss_function_around_one,
                     reference_currencies=all_currencies):
    starting_point = np.ones(len(selected_currencies))/len(selected_currencies)
    bounds = [(0, 1) for _ in range(len(starting_point))]
    
    result = minimize(fun=loss_function, 
                      x0=starting_point, 
                      args=(selected_currencies, reference_currencies),
                      bounds=bounds,
                      options={"disp": True, 
                               "maxiter": max_iter})
    
    output = pd.Series(result.x/result.x.sum(), index=reference_date_exhange_rate.loc[selected_currencies].index)
    output = output.sort_values(ascending=False)
        
    return output

In [28]:
duca_mix = pd.read_csv("duca_mix.csv", sep=";").set_index("currency")["weight"]
duca_mix_currencies = list(duca_mix.index)

In [ ]:
outputs = {}
diffs = {}

for start_date in pd.date_range(start="2005-04-01", periods=40, freq='3MS'):
    print(f"Evaluating Duca mix for start_date {start_date}")
    train, test, dev, val = split_data(df, start_date, test_start, validation_start, validation_end)
    reference_date_exhange_rate = train[more_stable_currencies].iloc[0]
    
    # Create exchange table for each currency
    exchange_table = dict()

    for base_currency in all_currencies:
        exchange_table[base_currency] = 1/train[all_currencies].divide(train[base_currency], axis=0)

    outputs[start_date] = run_optimization(selected_currencies=duca_mix_currencies, 
                                           loss_function=calculate_loss_function_around_one,
                                           reference_currencies=more_stable_currencies)
    
    diffs[start_date] = (outputs[start_date]-duca_mix).loc[duca_mix.index]
    
    print(f"Duca mix:\n{outputs[start_date]}\nDiff:\n{diffs[start_date]}\n")

Evaluating Duca mix for start_date 2005-04-01 00:00:00
Duca mix:
USD   0.28881
EUR   0.20059
JPY   0.14826
GBP   0.12479
KRW   0.10855
CNY   0.10507
NZD   0.02395
dtype: float64
Diff:
currency
USD    0.00001
EUR   -0.00001
JPY   -0.00004
GBP   -0.00001
KRW    0.00005
CNY   -0.00003
NZD    0.00005
dtype: float64

Evaluating Duca mix for start_date 2005-07-01 00:00:00
Duca mix:
USD   0.31473
EUR   0.18217
JPY   0.14932
GBP   0.12584
KRW   0.10522
CNY   0.09325
NZD   0.02949
dtype: float64
Diff:
currency
USD    0.02593
EUR   -0.01843
JPY    0.00102
GBP    0.00104
KRW   -0.00328
CNY   -0.01185
NZD    0.00559
dtype: float64

Evaluating Duca mix for start_date 2005-10-01 00:00:00
Duca mix:
USD   0.33002
EUR   0.17515
JPY   0.14471
GBP   0.13164
KRW   0.09501
CNY   0.08721
NZD   0.03626
dtype: float64
Diff:
currency
USD    0.04122
EUR   -0.02545
JPY   -0.00359
GBP    0.00684
KRW   -0.01349
CNY   -0.01789
NZD    0.01236
dtype: float64

Evaluating Duca mix for start_date 2006-01-01 00:00:00
Duc

In [ ]:
pd.concat([v.to_frame().reset_index() for v in diffs.values()]).groupby("currency").median()

In [ ]:
output_medians = (pd.concat([v.to_frame().reset_index() for v in outputs.values()])
                    .groupby("currency")
                    .median())

output_medians

In [ ]:
final_medians = output_medians / output_medians.sum()
final_medians

In [ ]:
final_medians.sum()

In [ ]:
(final_medians * 100).map(lambda x: round(x, 2))

In [ ]:
(final_medians * 100).map(lambda x: round(x, 2)).sum()